In [1]:
import torch

In [6]:
state_dict = torch.load('model_weights.pth')

In [7]:
state_dict.keys()

odict_keys(['net.linear0.weight', 'net.linear0.bias', 'net.batchnorm0.weight', 'net.batchnorm0.bias', 'net.batchnorm0.running_mean', 'net.batchnorm0.running_var', 'net.batchnorm0.num_batches_tracked', 'net.linear1.weight', 'net.linear1.bias', 'net.batchnorm1.weight', 'net.batchnorm1.bias', 'net.batchnorm1.running_mean', 'net.batchnorm1.running_var', 'net.batchnorm1.num_batches_tracked', 'net.linear_output.weight', 'net.linear_output.bias'])

In [8]:
import json

# Sample list
my_list = [1, 2, 3, 4, 5]

# File path to save the list
file_path = 'list_data.json'

# Writing the list to a JSON file
with open(file_path, 'w') as file:
    json.dump(my_list, file)

print("List saved successfully!")


List saved successfully!


In [10]:
# Reading the list from the JSON file
with open(file_path, 'r') as file:
    my_list2 = json.load(file)

print("List loaded successfully:", my_list)

List loaded successfully: [1, 2, 3, 4, 5]


In [1]:
"""Unit testing for the original SoftAdapt variant."""

from softadapt import SoftAdapt
import torch
import unittest

class TestSoftAdapt(unittest.TestCase):
    """Class for testing our finite difference implementation."""

    @classmethod
    def setUpClass(class_):
        class_.decimal_place = 5

    # First starting with positive slope test cases.
    def test_beta_positive_three_components(self):
        loss_component1 = torch.tensor([1, 2, 3, 4, 5])
        loss_component2 = torch.tensor([150, 100, 50, 10, 0.1])
        loss_component3 = torch.tensor([1500, 1000, 500, 100, 1])

        solutions = torch.tensor([9.9343e-01, 6.5666e-03, 3.8908e-22])

        softadapt_object = SoftAdapt(beta=0.1)
        alpha_0, alpha_1, alpha_2 = softadapt_object.get_component_weights(
                                                                loss_component1,
                                                                loss_component2,
                                                                loss_component3,
                                                                verbose=False)
        self.assertAlmostEqual(
            alpha_0.numpy(),
            solutions[0].numpy(),
            self.decimal_place,
            ("Incorrect SoftAdapt calculation for simple 'dominant loss' case."
             "The first loss component failed.")
        )

        self.assertAlmostEqual(
            alpha_1.numpy(),
            solutions[1].numpy(),
            self.decimal_place,
            ("Incorrect SoftAdapt calculation for simple 'dominant loss' case."
             "The second loss component failed.")
        )

        self.assertAlmostEqual(
            alpha_2.numpy(),
            solutions[2].numpy(),
            self.decimal_place,
            ("Incorrect SoftAdapt calculation for simple 'dominant loss' case."
             "The second loss component failed.")
        )


    # TODO: Add more sophisticated unit tests


# if __name__ == "__main__":
    # unittest.main()

In [2]:
unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument '/home/jxxiong/.local/share/jupyter/runtime/kernel-v2-1283026VOOvxDbgkzDC.json'


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [22]:
# loss_component1 = torch.tensor([1, 2, 3, 4, 2])
# loss_component2 = torch.tensor([0.1, 0.11, 0.9, 0.89, 0.84])
# loss_component3 = torch.tensor([1500, 1000, 500, 100, 1])

loss_component1 = torch.tensor([4, 5])
loss_component2 = torch.tensor([0.89, 0.84])
loss_component3 = torch.tensor([10, 1])

solutions = torch.tensor([9.9343e-01, 6.5666e-03, 3.8908e-22])

softadapt_object = SoftAdapt(beta=0.05)
alpha_0, alpha_1, alpha_2 = softadapt_object.get_component_weights(
                                                                loss_component1,
                                                                loss_component2,
                                                                loss_component3,
                                                                verbose=False)

In [23]:
[alpha_0, alpha_1, alpha_2]

[tensor(0.3913, dtype=torch.float64),
 tensor(0.3713, dtype=torch.float64),
 tensor(0.2374, dtype=torch.float64)]

In [2]:
import torch
from autograd_minimize import minimize
import numpy as np

def rosen_torch(x, a):
    return (a*(x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0).sum()
    
res = minimize(rosen_torch, np.array([0.,0.]), backend='torch')
print(res.x)

[0.99999912 0.99999824]


In [7]:
import torch
from torch.optim import LBFGS

# Define the system of equations as a function
def equations(x):
    # Example system: x^2 + y^2 = 4, x - y = 1
    return [x[0]**2 + x[1]**2 - 4, x[0] - x[1] - 1]

# Initial guess
x = torch.tensor([1.0, 0.0], requires_grad=True)

# Define the optimizer
optimizer = LBFGS([x], lr=1, max_iter=20, tolerance_grad=1e-7, tolerance_change=1e-9, history_size=100)

# Optimization loop
def closure():
    optimizer.zero_grad()
    eqs = equations(x)
    loss = sum((eq ** 2).sum() for eq in eqs)  # Sum of squares of the equations
    loss.backward()
    return loss

for i in range(100):
    optimizer.step(closure)
    # Check convergence by evaluating the loss without gradient calculations
    with torch.no_grad():
        current_loss = sum((eq ** 2).sum() for eq in equations(x))
        if current_loss < 1e-5:
            break

print(f"Solution: {x.detach().numpy()}")


Solution: [1.8228748 0.8228736]


In [13]:
import torch
from torch.optim import LBFGS

# Define the system of equations as a function with batch support
def batched_equations(x, a):
    # x: [batch_size, 2], a: [batch_size, 2]
    # Equations: x^2 + y^2 = a[:, 0], x - y = a[:, 1]
    return torch.stack([x[:, 0]**2 + x[:, 1]**2 - a[:, 0], x[:, 0] - x[:, 1] - a[:, 1]], dim=1)

# Batch of parameters and initial guesses
a_batch = torch.tensor([[4.0, 1.0], [5.0, 0.0]])  # Two sets of parameters
x_batch = torch.tensor([[1.0, 0.0], [2.0, 1.0]], requires_grad=True)  # Corresponding initial guesses

# Define the optimizer
optimizer = LBFGS([x_batch], lr=1, max_iter=20, tolerance_grad=1e-7, tolerance_change=1e-9, history_size=100)

# Optimization loop
def closure():
    optimizer.zero_grad()
    eqs = batched_equations(x_batch, a_batch)
    loss = (eqs ** 2).sum()  # Sum of squares of the residuals for all systems
    loss.backward()
    return loss

for i in range(5):
    optimizer.step(closure)
    with torch.no_grad():
        current_loss = (batched_equations(x_batch, a_batch) ** 2).sum()
        if current_loss < 1e-5:
            break

print(f"Solutions: {x_batch.detach().numpy()}")


Solutions: [[1.822875  0.8228766]
 [1.5811387 1.5811392]]


In [14]:
import torch
import torch.nn as nn
from torch.optim import LBFGS, Adam

# Neural Network Definition
class ParameterNetwork(nn.Module):
    def __init__(self):
        super(ParameterNetwork, self).__init__()
        # Simple network with one hidden layer
        self.fc1 = nn.Linear(1, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 2)  # Outputting 2 values for a

    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Initialize the neural network
param_net = ParameterNetwork()

# Example input tensor
z = torch.tensor([[1.0], [2.0]], requires_grad=True)  # Example input to the neural network

# Define optimizers
nn_optimizer = Adam(param_net.parameters(), lr=0.01)

def solve_system(a):
    # Initial guess for x
    x = torch.zeros(a.shape, requires_grad=True)
    
    # Define the optimizer for x
    x_optimizer = LBFGS([x], lr=1, max_iter=20, tolerance_grad=1e-7, tolerance_change=1e-9, history_size=100)

    # Define closure for LBFGS optimization
    def closure():
        x_optimizer.zero_grad()
        eqs = batched_equations(x, a)
        loss = (eqs ** 2).sum()
        loss.backward()
        return loss

    # Optimize x
    for i in range(100):
        x_optimizer.step(closure)
    
    return x

# Training loop
for epoch in range(100):
    nn_optimizer.zero_grad()

    # Forward pass through the network
    a = param_net(z)

    # Solve the system for x based on a
    x = solve_system(a)

    # Define a loss function for x (dummy example: minimizing the sum of x values)
    loss = x.sum()

    # Backward pass
    loss.backward()

    # Update the neural network
    nn_optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [15]:
import torch
import torch.nn as nn
from torch.optim import LBFGS, Adam

# Define the system of equations as a function with batch support
def batched_equations(x, a):
    # x: [batch_size, 2], a: [batch_size, 2]
    # Equations: x^2 + y^2 = a[:, 0], x - y = a[:, 1]
    return torch.stack([x[:, 0]**2 + x[:, 1]**2 - a[:, 0], x[:, 0] - x[:, 1] - a[:, 1]], dim=1)

# Neural Network Definition
class ParameterNetwork(nn.Module):
    def __init__(self):
        super(ParameterNetwork, self).__init__()
        # Simple network with one hidden layer
        self.fc1 = nn.Linear(1, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 2)  # Outputting 2 values for a

    def forward(self, z):
        out = self.fc1(z)
        out = self.relu(out)
        out = self.fc2(out)
        return out


# Solve the system of equations for x given parameters a
def solve_system(a, optimizer):
    # Initial guess for x
    x = torch.zeros(a.shape, requires_grad=True)

    # Define the optimizer for x
    x_optimizer = optimizer

    # Define closure for LBFGS optimization
    def closure():
        if x.grad is not None:
            x.grad.detach_()
            x.grad.zero_()
        eqs = batched_equations(x, a)
        loss = (eqs ** 2).sum()
        loss.backward()
        return loss

    # Optimize x
    x_optimizer.step(closure)

    return x  # Do not detach x here as we need gradients for backpropagation

# Initialize the neural network
param_net = ParameterNetwork()

# Example input tensor (could be your dataset or batch of data)
z = torch.tensor([[1.0], [2.0]], requires_grad=True)  # Example input to the neural network

# Define optimizers
nn_optimizer = Adam(param_net.parameters(), lr=0.01)

# Define optimizer for x outside the loop to reuse it
x_optimizer = LBFGS([torch.zeros_like(z, requires_grad=True)], lr=1, max_iter=20, tolerance_grad=1e-7, tolerance_change=1e-9, history_size=100)

# Training loop
for epoch in range(100):
    nn_optimizer.zero_grad()

    # Forward pass through the network
    a = param_net(z)

    # Solve the system for x based on a
    x = solve_system(a, x_optimizer)

    # Define a loss function for x (dummy example: minimizing the sum of x values)
    loss = x.sum()

    # Backward pass
    loss.backward()  # The gradients will now flow correctly

    # Update the neural network
    nn_optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")



RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn